# Habituation to a toy model of odor background
Look at new odor recognition on top of a two-odor, one fluctuating proportion background:

$$ \vec{x}(t) = \left(\frac12 + \nu \right) \vec{x}_a + \left(\frac12 - \nu \right) \vec{x}_b $$

The odor vectors are generated with i.i.d. exponential elements with scale (mean) mu=0.1, then normalized to have an L2 norm of 1. I take $\nu(t)$ following a Ornstein-Uhlenbeck process, typically with standard deviation $\sigma = 0.3$ and average $\langle \nu \rangle = 0$. 

Here, I optionally clip elements of $\vec{x}(t)$ to be non-negative, although this does not make a significant difference since samples where $|\nu| > 0.5$ are rare. 

## Imports

In [ ]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os, sys
if ".." not in sys.path:
    sys.path.insert(1, "..")

from modelfcts.ibcm import (
    integrate_inhib_ibcm_network_options,
    ibcm_respond_new_odors,
    compute_mbars_cgammas_cbargammas,
    ibcm_respond_new_odors
)
from modelfcts.ibcm_analytics import (
    fixedpoints_barm_2vectors, 
    fixedpoints_w_2vectors, 
    fixedpoint_s_2vectors_instant, 
)
from modelfcts.biopca import (
    integrate_inhib_biopca_network_skip,
    build_lambda_matrix,
    biopca_respond_new_odors
)
from modelfcts.average_sub import (
    integrate_inhib_average_sub_skip, 
    average_sub_respond_new_odors
)
from modelfcts.ideal import(
    find_projector, 
    find_parallel_component, 
    ideal_linear_inhibitor, 
    compute_ideal_factor_toy
)
from modelfcts.checktools import (
    analyze_pca_learning
)
from modelfcts.backgrounds import (
    update_ou_2inputs,
    generate_odorant
)
from modelfcts.tagging import (
    project_neural_tag, 
    create_sparse_proj_mat, 
    SparseNDArray, 
)
from utils.statistics import seed_from_gen
from utils.smoothing_function import (
    moving_average, 
    moving_var
)
from simulfcts.plotting import (
    plot_cbars_gamma_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition, 
    plot_pca_results, 
    hist_outline
)
from simulfcts.analysis import compute_back_reduction_stats
from utils.metrics import jaccard

In [ ]:
from utils.metrics import l2_norm, l1_norm, linf_norm, cosine_dist

def distance_panel_target(mixes, target):
    """ Compute a panel of distances between the pure (target) new odor and mixtures 
    (which can be without inhibition, with average inhibition, IBCM inhibition, etc.). 
    
    Four distances included, in order: l2, l1, linf, cosine_dist
    
    Args:
        mixes (np.ndarray): mixtures of odors to compute distance from target, 
            the last axis should have the size of target, 
            while other axes are arbitrary.  
        target (np.1darray): target odor vector, same length as
            last axis of mixes. 
    Returns:
        dist_panel (np.ndarray): shape of pure, except the last axis, 
            which has length 4 (for the number of distances computed). 
    """
    # Make axis 0 the axis indexing distance metrics, to begin with
    # And move it to the last axis before returning
    dist_array = np.zeros([4] + list(mixes.shape[:-1]))
    # No need to add axes to target vector; if it is 1d, it is broadcasted
    # along the last axis of mixes, which indexes elements of each vector. 
    dist_array[0] = l2_norm(target - mixes)
    dist_array[1] = l1_norm(target - mixes)
    dist_array[2] = linf_norm(target - mixes)
    dist_array[3] = cosine_dist(target, mixes)
    
    return np.moveaxis(dist_array, 0, -1)

### Aesthetic parameters

In [ ]:
#plt.style.use(['dark_background'])
plt.rcParams["figure.figsize"] = (4.5, 3.0)

In [ ]:
models = ["ibcm", "biopca", "avgsub", "ideal", "orthogonal", "none"]
model_nice_names = {
    "ibcm": "IBCM",
    "biopca": "BioPCA",
    "avgsub": "Average",
    "ideal": "Ideal",
    "orthogonal": "Orthogonal",
    "none": "None"
}
model_colors = {
    "ibcm": "xkcd:turquoise",
    "biopca": "xkcd:orangey brown",
    "avgsub": "xkcd:navy blue",
    "ideal": "xkcd:powder blue",
    "orthogonal": "xkcd:pale rose",
    "none": "grey"
}

### Initialization

In [ ]:
# Initialize common simulation parameters
n_dimensions = 25
n_components = 2  # Two odors but only one fluctuating proportion

inhib_rates = [0.00025, 0.00005]  # alpha, beta

# Simulation duration
duration = 80000.0
deltat = 1.0
skp = 1  # We can save all time steps, small enough network

# Common model options
activ_function = "identity"  #"ReLU"

# Background process
update_fct = update_ou_2inputs

# Choose randomly generated background vectors
rgen_meta = np.random.default_rng(seed=0x4146e7d68791560a406669280c37e5bb)

back_components = np.zeros([n_components, n_dimensions])
for i in range(n_components):
    back_components[i] = generate_odorant(n_dimensions, rgen_meta, lambda_in=0.1)
back_components = back_components / l2_norm(back_components).reshape(-1, 1)

# Seed for background simulation, to make sure all models are the same
simul_seed = seed_from_gen(rgen_meta)

# Initial background vector and initial nu values
average_nu = np.zeros(1)
init_nu = np.zeros(1)
init_bkvec = 0.5*back_components[0] + 0.5*back_components[1]
# nus are first in the list of initial background params
init_back_list = [init_nu, init_bkvec]

## Compute the coefficients in the Ornstein-Uhlenbeck update equation
sigma2_nu = 0.09
tau_nu = 2.0  # Fluctuation time scale of the background nu_alphas (same for all)
update_coefs_mean = np.exp(-deltat/tau_nu)
update_coefs_noise = np.sqrt(sigma2_nu*(1 - np.exp(-2*deltat/tau_nu)))

back_params = [average_nu, update_coefs_mean, update_coefs_noise, back_components]

## IBCM habituation
### IBCM simulation

In [ ]:
# IBCM model parameters
n_i_ibcm = 2  # Two neurons is enough, one per fixed point x_{\pm}

learnrate_ibcm = 0.0025
tau_avg_ibcm = 300
coupling_eta_ibcm = 0.2
decay_relative_ibcm = 0.005
lambd_ibcm = 1.0
ibcm_rates = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    None,   # Saturation, none
    None,   # For the Law and Cooper variant
    decay_relative_ibcm
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "linear", 
    "variant": "intrator", 
    "decay": True
}

# Initial synaptic weights: small positive noise
init_synapses_ibcm = 0.1*rgen_meta.random(size=[n_i_ibcm, n_dimensions])

In [ ]:
# Run the IBCM simulations
simul_seed = seed_from_gen(rgen_meta)
sim_results = integrate_inhib_ibcm_network_options(
                init_synapses_ibcm, update_fct, init_back_list, 
                ibcm_rates, inhib_rates, back_params, duration, 
                deltat, seed=simul_seed, noisetype="normal",  
                skp=skp, **ibcm_options
)

(tser_ibcm, 
 nuser_ibcm, 
 bkvecser_ibcm, 
 mser_ibcm, 
 cbarser_ibcm, 
 thetaser_ibcm,
 wser_ibcm, 
 yser_ibcm) = sim_results

### IBCM habituation analysis

In [ ]:
# Calculate cgammas_bar and mbars
transient = 50000 // skp
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbarser_ibcm, c_gammas, cbars_gamma = compute_mbars_cgammas_cbargammas(
                                    mser_ibcm, coupling_eta_ibcm, back_components)
sums_cbars_gamma = np.sum(cbars_gamma, axis=2)
sums_cbars_gamma2 = np.sum(cbars_gamma*cbars_gamma, axis=2)

# Analytical prediction of fixed points
analytical_barm, analytical_sign_labels = fixedpoints_barm_2vectors(back_components, np.sqrt(sigma2_nu), 
                                               coupling_eta_ibcm, n_r=n_dimensions, lambd=lambd_ibcm)
# Determine which fixed point the system is at
mdots_signs = np.sign(mbarser_ibcm[-1].dot(back_components[0] - back_components[1]))
sign_map = {1:"+", -1:"-"}
mdots_signs = "(" + ", ".join(map(lambda x: sign_map.get(x), mdots_signs)) + ")"
sign_idx = analytical_sign_labels.index(mdots_signs)

# Plot dot products with x_a and x_b, in terms of reduced m vectors. 
# Take any of the fixed points for one neuron, its two dot products are the only two possible dot product values. 
c_specif = np.dot(analytical_barm[sign_idx, 0], back_components[0])
c_nonspecif = np.dot(analytical_barm[sign_idx, 0], back_components[1])

# Analytical predictions of W and s
# rates, fixed_mbar, bk_components, sigma2
ibcm_w_pred = fixedpoints_w_2vectors(inhib_rates, analytical_barm[sign_idx, [0, 1]], back_components, sigma2_nu) 
ibcm_yser_pred = fixedpoint_s_2vectors_instant(inhib_rates, bkvecser_ibcm, options=ibcm_options)
analytical_factor = inhib_rates[1] / (2*inhib_rates[0] + inhib_rates[1])

In [ ]:
# Plot the cbar2_avg term throughout
cbar2_avg_ser = moving_average(cbarser_ibcm*cbarser_ibcm, kernelsize=tau_avg_ibcm)
neurons_cmap = sns.color_palette("Greys", n_colors=n_i_ibcm)
fig, ax = plt.subplots()
for i in range(n_i_ibcm):
    ax.plot(tser_ibcm[:-tau_avg_ibcm], cbar2_avg_ser[:-tau_avg_ibcm, i], 
            color=neurons_cmap[i])
ax.set(xlabel="Time (x1000)", ylabel=r"$\bar{c}^2$ moving average")
plt.show()
plt.close()

In [ ]:
fig , ax, _ = plot_cbars_gamma_series(tser_ibcm, cbars_gamma, 
                        skp=10, transient=50000 // skp)
# Compare to exact analytical fixed point solution
ax.axhline(c_specif, ls="--", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{specific}}$")
ax.axhline(c_nonspecif, ls="-.", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{non}}$")
ax.legend()
plt.show()
plt.close()

In [ ]:
# Correlation between nu's and c's, see if some neurons are specific to odors
# Each neuron turns out to correlate its response to  one concentration
# that means it is specific to that odor. 
cbarser_norm_centered = cbarser_ibcm - np.mean(cbarser_ibcm[transient:], axis=0)
conc_ser_centered = np.stack([nuser_ibcm[:, 0], -nuser_ibcm[:, 0]], axis=1)
correl_c_nu = np.mean(cbarser_norm_centered[transient:, :, None] 
                      * conc_ser_centered[transient:, None, :], axis=0)

fig, ax = plt.subplots()
img = ax.imshow(correl_c_nu.T)
ax.set(ylabel=r"Component $\gamma$", xlabel=r"Neuron $i$")
fig.colorbar(img, label=r"$\langle (\bar{c}^i - \langle \bar{c}^i \rangle)"
             r"(\nu_{\gamma} - \langle \nu_{\gamma} \rangle) \rangle$", 
            location="top")
plt.show()
plt.close()

# Check if each component has at least one neuron
split_val = 0.0
for comp in range(n_components):
    print("Number of neurons specific to component {}: {}".format(
            comp, np.sum(np.mean(cbars_gamma[-2000:, :, comp], axis=0) > split_val)))

In [ ]:
fig, axes, _ = plot_background_neurons_inhibition(tser_ibcm, bkvecser_ibcm, yser_ibcm, skp=10)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                tser_ibcm, bkvecser_ibcm, yser_ibcm, skp=10)

# Compute noise reduction factor, annotate
transient = 50000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_ibcm, wser_ibcm, skp=100)
for j in range(axes.size):
    for i in range(n_dimensions):
        axes.flat[j].axhline(ibcm_w_pred[j, i], color="grey", zorder=-i, lw=1.0)
fig.set_size_inches(*[1.5*a for a in fig.get_size_inches()])
plt.show()
plt.close()

In [ ]:
# Second moment of s, simulation vs prediction
sts_ibcm_ser = np.sum(yser_ibcm*yser_ibcm, axis=1)
sts_ibcm_pred = np.sum(ibcm_yser_pred*ibcm_yser_pred, axis=1)
fig, axes = plt.subplots(2, sharex=True)
axes = axes.flatten()
axes[0].plot(tser_ibcm/1000, sts_ibcm_ser, color="k")
axes[0].plot(tser_ibcm/1000, sts_ibcm_pred, color="blue", alpha=0.9)
axes[0].set(ylabel=r"$\vec{s}^T \vec{s}$")
axes[0].set_ylim([-0.01, 0.1])
# Plot the difference
axes[1].plot(tser_ibcm/1000, sts_ibcm_pred - sts_ibcm_ser, color="k")
axes[1].set(xlabel="Time (x1000 steps)", ylabel=r"Pred. $-$ sim. $\vec{s}^T \vec{s}$")
axes[1].set_ylim([-0.1, 0.05])
plt.show()
plt.close()

### Export IBCM simulation results for plotting
Export background concentration time series, IBCM time series of $M$ and $W$. 

Maybe PCA time series for supplementary plots. 

## BioPCA simulation
### BioPCA habituation simulation

In [ ]:
# BioPCA model parameters
n_i_pca = 1  # Number of inhibitory neurons for BioPCA case

# Model rates
learnrate_pca = 0.0005  # Learning rate of M
# Choose Lambda diagonal matrix as advised in Minden et al., 2018
lambda_range_pca = 0.5
lambda_max_pca = 5.0
lambda_mat_diag = build_lambda_matrix(lambda_max_pca, lambda_range_pca, n_i_pca)
# Learning rate of L, relative to learnrate. Adjusted to Lambda in the integration function
rel_lrate_pca = 2.0  #  / lambda_max_pca**2 
xavg_rate_pca = learnrate_pca
pca_options = {
    "activ_fct": activ_function, 
    "remove_lambda": False, 
    "remove_mean": True
}
biopca_rates = [learnrate_pca, rel_lrate_pca, lambda_max_pca, lambda_range_pca]
if pca_options["remove_mean"]:
    biopca_rates.append(xavg_rate_pca)


# Initial synaptic weights: small positive noise
init_synapses_pca = rgen_meta.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_i_pca)
init_mmat_pca = rgen_meta.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_dimensions)
init_lmat_pca = np.eye(n_i_pca, n_i_pca)  # Supposed to be near-identity, start as identity
ml_inits_pca = [init_mmat_pca, init_lmat_pca]

In [ ]:
# Run simulation
sim_results = integrate_inhib_biopca_network_skip(
                ml_inits_pca, update_fct, init_back_list, biopca_rates, 
                inhib_rates, back_params, duration, deltat, 
                seed=simul_seed, noisetype="normal", skp=skp, **pca_options)
(tser_pca, 
 nuser_pca, 
 bkvecser_pca, 
 mser_pca, 
 lser_pca, 
 xser_pca, 
 cbarser_pca, 
 wser_pca, 
 yser_pca) = sim_results

### BioPCA simulation analysis

#### Analytical predictions for one PCA neuron
$L$ matrix: 1x1, a scalar, equal to the inverse of the eigenvalue: $L = 1/L' = \frac{1}{\sigma^2 \| \vec{x}_s \|^2}$

$M$ matrix: 1x$n_R$, parallel to the fluctuating part of the background, $\vec{m} = \sigma^2 \| \vec{x}_s \| \vec{x}_s$.

$W$ matrix: $n_Rx1$, a vector also parallel to $\vec{x}_s$, we find $\vec{w} = \frac{\sigma^2 \| \vec{x}_s \|}{\sigma^2 \| \vec{x}_s \|^2 + \beta/\alpha} \vec{x}_s$. 

Then the instantaneous PN activity should be $\vec{s}(t) = \vec{x}(t) - \langle \vec{x} \rangle - WLM(\vec{x}(t) - \langle \vec{x} \rangle) = \frac{\beta/\alpha}{\beta/\alpha + \sigma^2 \| \vec{x}_s \|^2} \nu(t) \vec{x}_s$

In [ ]:
from modelfcts.pca_analytics import (
    fixedpoints_pca_2vectors, 
    pca_fixedpoint_s_2vectors_instant, 
    pca_fixedpoint_s_2vectors_variance
)

In [ ]:
# Analytical prediction
x_s = (back_components[0] - back_components[1])
x_d = (back_components[0] + back_components[1])/2
m_pca_pred, l_pca_pred, w_pca_pred = fixedpoints_pca_2vectors(back_components, 
                                        sigma2_nu, inhib_rates, lambd=lambda_max_pca)
yser_pca_pred = pca_fixedpoint_s_2vectors_instant(back_components, sigma2_nu, 
                                    inhib_rates, bkvecser_pca, lambd=lambda_max_pca)
yvari_pca_pred = pca_fixedpoint_s_2vectors_variance(back_components,
                                sigma2_nu, inhib_rates, lambd=lambda_max_pca)

# Analysis of simulation
res = analyze_pca_learning(bkvecser_pca, mser_pca, lser_pca, 
                           lambda_mat_diag, demean=pca_options["remove_mean"])
true_pca, learnt_pca, fser, off_diag_l_avg_abs, align_error_ser = res

In [ ]:
print(m_pca_pred)
print(l_pca_pred)
print(w_pca_pred)
print(yser_pca_pred[-1])
xvari_pca = np.var(l2_norm(bkvecser_pca, axis=1))
xvari_pca = np.mean(np.sum((bkvecser_pca - x_d)**2, axis=1), axis=0)
print(np.sqrt(yvari_pca_pred / xvari_pca))

In [ ]:
fig, axes = plot_pca_results(tser_pca/1000, true_pca, learnt_pca, align_error_ser, off_diag_l_avg_abs)
axes[-1].set_xlabel("Time (x1000 steps)")
fig.set_size_inches(fig.get_size_inches()[0], 3*2)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                tser_pca, bkvecser_pca, yser_pca, skp=10)

# Compute noise reduction factor, annotate
transient = 50000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
fig, axes, _ = plot_background_neurons_inhibition(tser_pca, bkvecser_pca, yser_pca, skp=10)
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_pca, wser_pca, skp=10)
for i in range(n_dimensions):
    axes.flat[0].axhline(w_pca_pred[0, i], ls="--", zorder=-i, lw=0.5)
fig.set_size_inches(plt.rcParams["figure.figsize"])
plt.show()
plt.close()

In [ ]:
# Second moment of s, simulation vs prediction
sts_pca_ser = np.sum(yser_pca*yser_pca, axis=1)
sts_pca_pred = np.sum(yser_pca_pred*yser_pca_pred, axis=1)
fig, axes = plt.subplots(2, sharex=True)
axes = axes.flatten()
axes[0].plot(tser_pca/1000, sts_pca_ser, color="k")
axes[0].plot(tser_pca/1000, sts_pca_pred, color="blue", alpha=0.9)
axes[0].set(ylabel=r"$\vec{s}^T \vec{s}$")
# Plot the difference
axes[1].plot(tser_pca/1000, sts_pca_pred - sts_pca_ser, color="k")
axes[1].set(xlabel="Time (x1000 steps)", ylabel=r"Pred. $-$ sim. $\vec{s}^T \vec{s}$")
plt.show()
plt.close()

### Export BioPCA simulation for plotting
We have a good analytical understanding of BioPCA's W weights on the toy model. Show that in a supplementary figure too. Maybe combine with the toy model figure of IBCM? TBD. 

In [ ]:
results_filename = os.path.join("..", "results", "for_plots", "sample_2d_simulation_biopca.npz")
save_skp = 10
np.savez_compressed(
    results_filename, 
    tser=tser_pca[::save_skp], 
    nuser=nuser_pca[::save_skp], 
    mser=mser_pca[::save_skp],
    m_pred=m_pca_pred,
    lser=lser_pca[::save_skp],
    l_pred=l_pca_pred,
    wser=wser_pca[::save_skp], 
    w_pred=w_pca_pred,
    yser=yser_pca[::save_skp], 
    yser_pred=yser_pca_pred[::save_skp],
    yvari_pred=yvari_pca_pred,
    true_pca_vals=true_pca[0],
    true_pca_vecs=true_pca[1], 
    learnt_pca_vals=learnt_pca[0][::save_skp],
    learnt_pca_vecs=learnt_pca[1][::save_skp],
    align_error=align_error_ser[::save_skp],
    back_vecs=back_components,
    save_skp=np.ones(1, dtype=int)*save_skp
)

### Average background subtraction simulation

In [ ]:
# Average subtraction model parameters
avg_options = {"activ_fct": activ_function}

# Initial synaptic weights: dummy
init_synapses_avg = np.zeros([1, n_dimensions])

In [ ]:
sim_results = integrate_inhib_average_sub_skip(
                init_synapses_avg, update_fct, init_back_list, 
                [], inhib_rates, back_params, duration, deltat,
                seed=simul_seed, noisetype="normal", skp=skp, **avg_options
)
tser_avg, bkser_avg, bkvecser_avg, wser_avg, yser_avg = sim_results

### Ideal inhibition
The component parallel to the background is reduced to beta / (2*alpha + beta). 

In [ ]:
back_projector = find_projector(back_components.T)
ideal_factor = inhib_rates[1] / (2*inhib_rates[0] + inhib_rates[1])
yser_ideal = bkvecser_ibcm * ideal_factor

## Model comparison for background inhibition

In [ ]:
ynorm_series = {
    "ibcm": l2_norm(yser_ibcm), 
    "biopca": l2_norm(yser_pca), 
    "avgsub": l2_norm(yser_avg), 
    "none": l2_norm(bkvecser_ibcm), 
    "ideal": l2_norm(yser_ideal)
}
std_options = dict(kernelsize=2001, boundary="free")
mean_options = dict(kernelsize=2001, boundary="free")
std_series = {
    a: np.sqrt(moving_var(ynorm_series[a], **std_options)) for a in ynorm_series
} 
mean_series = {
    a: moving_average(ynorm_series[a], **mean_options) for a in ynorm_series
}

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)
axes = axes.flatten()
for model in std_series.keys():
    props = dict(label=model_nice_names[model], color=model_colors[model])
    axes[0].plot(tser_ibcm / 1000, mean_series[model], **props)
    axes[1].plot(tser_ibcm / 1000, std_series[model], **props)
ynorm_string = r"$\|\vec{s}\|$"
axes[0].set_ylabel(r"PN activity norm, " + ynorm_string)
axes[1].set(xlabel="Time (x1000 steps)", ylabel=r"Standard deviation " + ynorm_string)
axes[0].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
fig.set_size_inches(4.5, 2.5*2)
plt.show()
plt.close()

## Model comparison for new odor recognition

In [ ]:
def find_snap_index(dt, skip, times):
    """ Find nearest multiple of dt*skip to each time in times """
    return np.around(times / (dt*skip)).astype(int)

In [ ]:
# Common parameters
n_kc = 1000
projection_arguments = {
    "kc_sparsity": 0.05,
    "adapt_kc": True,
    "n_pn_per_kc": 3,
    "project_thresh_fact": 0.1
}
proj_mat = create_sparse_proj_mat(n_kc, n_dimensions, rgen_meta)

yser_dict = {
    "ibcm": yser_ibcm, 
    "biopca": yser_pca, 
    "avgsub": yser_avg, 
    "none": bkvecser_ibcm, 
    "ideal": yser_ideal
}

In [ ]:
# Generate new odors, select test times, etc.
# New odors tested
n_new = 100
new_odors = generate_odorant([n_new, n_dimensions], rgen_meta, lambda_in=0.1)
new_odors /= l2_norm(new_odors)[:, None]

# Test times
n_test_times = 10
start_test_t = duration - n_test_times * 2000.0
test_times = np.linspace(start_test_t, duration, n_test_times)
test_times -= deltat*skp
test_idx = find_snap_index(deltat, skp, test_times)

# New odor concentrations
new_test_concs = np.asarray([0.5, 1.0])
avg_whiff_conc = 0.5
print("Average whiff concentration: {:.4f}".format(avg_whiff_conc))
new_test_concs *= avg_whiff_conc
n_new_concs = len(new_test_concs)

# Background samples, indexed [time, sample, n_orn]
n_back_samples = 10
# Steady-state distribution of nu: normal distribution, mean 0, variance sigma2
conc_samples = rgen_meta.normal(loc=0.0, scale=np.sqrt(sigma2_nu), size=n_test_times*(n_back_samples-1))
# Clip between -0.5 and 0.5
conc_samples = np.clip(conc_samples, a_min=-0.5, a_max=0.5).reshape(-1, 1)
back_samples = (0.5+conc_samples)*back_components[0:1] + (0.5-conc_samples)*back_components[1:2]
back_samples = back_samples.reshape([n_test_times, n_back_samples-1, -1])
back_samples = np.concatenate([bkvecser_ibcm[test_idx, None, :], back_samples], axis=1)

# Containers for s vectors of each model
mixture_yvecs = {a: np.zeros([n_new, n_test_times,  n_new_concs,  
                    n_back_samples, n_dimensions]) for a in yser_dict.keys()}
mixture_tags = {a: SparseNDArray((n_new, n_test_times, n_new_concs,
                    n_back_samples, n_kc), dtype=bool) for a in yser_dict.keys()}
new_odor_tags = sparse.lil_array((n_new, n_kc), dtype=bool)
jaccard_scores = {a: np.zeros([n_new, n_test_times, n_new_concs,  n_back_samples]) 
                  for a in yser_dict.keys()}

In [ ]:
# Compute ideal reduction factor for each concentration
dummy_rgen = np.random.default_rng(0x6e3e2886c30163741daaaf7c8b8a00e6)
ideal_factors = [compute_ideal_factor_toy(c, sigma2_nu, n_dimensions, 
                    generate_odorant, (dummy_rgen,)) for c in new_test_concs]
for i in range(n_new):
    # Compute neural tag of the new odor alone, without inhibition
    new_tag = project_neural_tag(
                    new_odors[i], new_odors[i],
                    proj_mat, **projection_arguments
                )
    new_odor_tags[i, list(new_tag)] = True
    # Parallel and orthogonal components
    x_new_par = find_parallel_component(new_odors[i], 
                        back_components, back_projector)
    x_new_ort = new_odors[i] - x_new_par
    # Now, loop over snapshots, mix the new odor with the back samples,
    # compute the PN response at each test concentration,
    # compute tags too, and save results
    for j in range(n_test_times):
        jj = test_idx[j]
        for k in range(n_new_concs):
            mixtures = (back_samples[j]
                + new_test_concs[k] * new_odors[i])
            # odors, mlx, wmat, 
            # Compute for each model
            mixture_yvecs["ibcm"][i, j, k] = ibcm_respond_new_odors(
                mixtures, mser_ibcm[jj], wser_ibcm[jj], 
                ibcm_rates, options=ibcm_options
            )
            mixture_yvecs["biopca"][i, j, k] = biopca_respond_new_odors(
                mixtures, [mser_pca[jj], lser_pca[jj], xser_pca[jj]], 
                wser_pca[jj], biopca_rates, options=pca_options
            )
            mixture_yvecs["avgsub"][i, j, k] = average_sub_respond_new_odors(
                mixtures, wser_avg[jj], options=avg_options
            )
            mixture_yvecs["none"][i, j, k] = mixtures
            mixture_yvecs["ideal"][i, j, k] = ideal_linear_inhibitor(
                x_new_par, x_new_ort, mixtures, new_test_concs[k], 
                ideal_factors[k], **avg_options
            )
            for l in range(n_back_samples):
                for mod in mixture_yvecs.keys():
                    mix_tag = project_neural_tag(
                        mixture_yvecs[mod][i, j, k, l], mixtures[l],
                        proj_mat, **projection_arguments
                    )
                    try:
                        mixture_tags[mod][i, j, k, l, list(mix_tag)] = True
                    except ValueError as e:
                        print(mix_tag)
                        print(mixture_yvecs[mod][i, j, k, l])
                        print(proj_mat.dot(mixture_yvecs[mod][i, j, k, l]))
                        raise e
                    jaccard_scores[mod][i, j, k, l] = jaccard(mix_tag, new_tag)

In [ ]:
# Plot model histogram results
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True)
fig.set_size_inches(9.5, 4)
axes = axes.flatten()
models = ["none", "ideal", "avgsub", "biopca", "ibcm"]
for m in models:  # Plot IBCM last
    all_jacs = jaccard_scores[m]
    for i in range(n_new_concs):
        hist_outline(
            axes[i], all_jacs[:, :, i, :].flatten(),
            bins="doane", density=True, label=model_nice_names.get(m, m),
            color=model_colors.get(m), alpha=1.0
        )
        axes[i].axvline(
            np.median(all_jacs[:, :, i, :]), ls="--",
            color=model_colors.get(m)
        )
# Labeling the graphs, etc.
for i in range(n_new_concs):
    ax = axes[i]
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i]))
    axes[i].set_xlabel("Jaccard similarity (higher is better)")
    axes[i].set_ylabel("Probability density")
axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
#fig.savefig("../figures/detection/compare_models_one_odor_habituation_{}.pdf".format(activ_function),
#            transparent=True, bbox_inches="tight")

plt.show()
plt.close()

In [ ]:
# Distance to new odor
# Plot model histogram results
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True)
fig.set_size_inches(9.5, 4)
axes = axes.flatten()
models = ["none", "ideal", "avgsub", "biopca", "ibcm"]
all_medians = []
for m in models:  # Plot IBCM last
    all_distances = (mixture_yvecs[m] 
         - new_test_concs[None, None, :, None, None]*new_odors[:, None, None, None, :])
    all_norms = l2_norm(all_distances.reshape(-1, n_dimensions))
    all_medians.append(np.median(all_norms))
    for i in range(n_new_concs):
        hist_outline(
            axes[i], all_norms,
            bins="doane", density=True, label=model_nice_names.get(m, m),
            color=model_colors.get(m), alpha=1.0
        )
        axes[i].axvline(
            all_medians[-1], ls="--",
            color=model_colors.get(m)
        )
# Labeling the graphs, etc.
for i in range(n_new_concs):
    axes[i].set_xlim([0.0, 2.0*max(all_medians)])
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i]))
    axes[i].set_xlabel(r"Distance to new odor, $\|\vec{s} - \vec{x}_{\mathrm{new}}\|$")
    axes[i].set_ylabel("Probability density")
axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
#fig.savefig("../figures/detection/compare_models_onerun_ynorm_{}.pdf".format(activ_fct),
#            transparent=True, bbox_inches="tight")

plt.show()
plt.close()

# Supplementary panels: convergence time
Considering one IBCM neuron, comparing analytical predictions to numerical simulations of how long it takes to reach steady-state. 

In [ ]:
from modelfcts.ibcm_analytics import find_convergence_time, analytical_convergence_times_2d
from modelfcts.ibcm import integrate_ibcm

In [ ]:
# Try the convergence prediction with the previous IBCM parameters
# but with a new simulation for a single neuron. Still with n_ORN=25
def example_convergence_time(input_vecs, learning_mu, tau_theta, moments_nu, tau_nu, rng):
    n_orn = input_vecs.shape[1]
    tmax = 160000
    dt = 1.0
    ## Recompute the coefficients in the Ornstein-Uhlenbeck update equation
    average_nu, sigma2_nu = moments_nu
    update_coefs_mean = np.exp(-dt/tau_nu)
    update_coefs_noise = np.sqrt(sigma2_nu*(1 - np.exp(-2*dt/tau_nu)))
    init_back_vec = np.sum(input_vecs, axis=0) / 2
    bk_update_params = [average_nu, update_coefs_mean, update_coefs_noise, back_components]
    
    # Show an example
    m_init = 0.025*rng.standard_normal(size=n_orn)
    simul_seed = seed_from_gen(rng)
    tser, mser, nuser, cser, _, bkvecser = integrate_ibcm(m_init, update_ou_2inputs, 
                        [np.zeros(1), init_back_vec], bk_update_params, tmax, dt, 
                        learnrate=learning_mu, seed=simul_seed, noisetype="normal", tavg=tau_theta)
    
    mdota_ser = np.dot(mser, input_vecs[0])
    mdotb_ser = np.dot(mser, input_vecs[1])
    mdotd_ser = 0.5*(mdota_ser + mdotb_ser)
    mdots_ser = mdota_ser - mdotb_ser

    initial_mdotds = [mdotd_ser[0], mdots_ser[0]]
    norms_vecds = [0.25*np.sum((input_vecs[0] + input_vecs[1])**2), 
                   np.sum((input_vecs[0] - input_vecs[1])**2)]
    predict_td, predict_ts = analytical_convergence_times_2d(initial_mdotds, 
                                        norms_vecds, learning_mu, sigma2_nu)
    
    mdotsers = np.stack([mdota_ser, mdotb_ser, mdotd_ser, mdots_ser])
    predicts_tds = np.asarray([predict_td, predict_ts])
    analytical_mdotab = np.asarray([1.0 + 1.0 / (2.0 * np.sqrt(sigma2_nu)), 
                                    1.0 - 1.0 / (2.0 * np.sqrt(sigma2_nu))])
    analytical_mdotds = np.asarray([1.0, 1.0 / np.sqrt(sigma2_nu), -1.0 / np.sqrt(sigma2_nu)])
    
    return tser, mdotsers, predicts_tds, analytical_mdotab, analytical_mdotds

In [ ]:
moments_conc = [average_nu, sigma2_nu]
rgen_time = np.random.default_rng(seed=0x2e56695e3b22bb595b16e645a2e9402)
res = example_convergence_time(back_components, learnrate_ibcm, 
                        tau_avg_ibcm, moments_conc, tau_nu, rgen_time)
tser_t, mdotsers, tdspreds, mdotabpreds, mdotdspreds = res

In [ ]:
# Plot the example
fig, axes = plt.subplots(1, 2)
axes = axes.flatten()
ax = axes[0]
# Plot dot products with vectors x_a and x_b
skpt = 20
ax.plot(tser_t[::skpt], mdotsers[0][::skpt], color="red", label=r"$\vec{m}(t) \cdot \vec{x}_a$")
ax.plot(tser_t[::skpt], mdotsers[1][::skpt], color="pink", label=r"$\vec{m}(t) \cdot \vec{x}_b$")

# Analytical predictions
ax.axhline(mdotabpreds[0], label=r"$1 + 1/(2 \sigma)$", ls="--", color="k")
ax.axhline(mdotabpreds[1], label=r"$1 - 1/(2 \sigma)$", ls="-.", color="k")
ax.set(xlabel="Time", ylabel=r"Dot products $\vec{m} \cdot \vec{x}_{a, b}$")

# Plot dot products with x_s and x_d
ax = axes[1]
ax.plot(tser_t[::skpt], mdotsers[3][::skpt], color="orange", label=r"$\vec{m}(t) \cdot \vec{x}_s$")
ax.plot(tser_t[::skpt], mdotsers[2][::skpt], color="blue", label=r"$\vec{m}(t) \cdot \vec{x}_d$")

ax.axhline(mdotdspreds[1], label=r"$\vec{m} \cdot \vec{x}_s = \pm 1/\sigma$", ls="-.", 
           color="xkcd:orangey brown")
ax.axhline(mdotdspreds[0], label=r"$\vec{m} \cdot \vec{x}_d = 1$", color="xkcd:marine", ls="--")
ax.set(xlabel="Time", ylabel=r"Dot products $\vec{m} \cdot \vec{x}_{s, d}$")
# Convergence times predicted
for ax in axes:
    ax.axvline(tdspreds[0], color="cyan", ls=":", label=r"Conv. time $c_d$")
    ax.axvline(tdspreds[1], color="orange", ls="--", label=r"Conv. time $c_s$")
    ax.legend(fontsize=8, loc='center right')

for ax in axes:
    ax.set_xticks([0, 40000, 80000, 120000, 160000])
fig.set_size_inches(7.5, 3.)
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
def scaling_convergence_time(*args, seed_sequence=None, n_tries=2):
    """ Check how ts and td scale as a function of initial x_d and x_s
    n_tries x n_tries values of initial x_s and x_d are tried
    Plot t_d - t_s, which should depend on eps_d only once this leading
    order behaviour of t_s is removed. 
    
    Same arguments as example_convergence_time, except the rng one. 
    """
    input_vecs, learning_mu, tau_theta, moments_nu, tau_nu = args
    
    # Random number generation business
    if seed_sequence is None:
        seed_sequence = np.random.SeedSequence()
    all_seeds = list(seed_sequence.spawn(n_tries*n_tries))
    
    # Initialize parameters
    n_orn = input_vecs.shape[1]
    tmax = 160000
    dt = 1.0
    ## Recompute the coefficients in the Ornstein-Uhlenbeck update equation
    average_nu, sigma2_nu = moments_nu
    update_coefs_mean = np.exp(-dt/tau_nu)
    update_coefs_noise = np.sqrt(sigma2_nu*(1 - np.exp(-2*dt/tau_nu)))
    init_back_vec = np.sum(input_vecs, axis=0) / 2
    bk_update_params = [average_nu, update_coefs_mean, update_coefs_noise, back_components]
    
    # Loop over pairs of x_s, x_d values
    epsd_axis = np.geomspace(0.02, 0.2, n_tries)*0.5
    epss_axis = np.geomspace(0.01, 0.1, n_tries)*0.5
    td_grid = np.zeros([2, n_tries, n_tries])  # Should only depend on epss
    ts_grid = np.zeros([2, n_tries, n_tries])  
    x_d, x_s = np.sum(input_vecs, axis=0)/2, input_vecs[0] - input_vecs[1]    
    norms_vecds = [np.sum(x_d**2), np.sum(x_s**2)]
    
    for i, epsd in enumerate(epsd_axis):
        for j, epss in enumerate(epss_axis):
            # Combine x_s and x_d to form initial m vector (x_s, x_d are orthogonal, so easy)
            m_init = epsd * x_d / norms_vecds[0] + epss * x_s / norms_vecds[1]
            tser, mser, nuser, cser, _, bkvecser = integrate_ibcm(m_init, update_ou_2inputs, 
                        [np.zeros(1), init_back_vec], bk_update_params, tmax, dt, 
                        learnrate=learning_mu, seed=all_seeds.pop(), noisetype="normal", tavg=tau_theta)
    
            mdota_ser = np.dot(mser, input_vecs[0])
            mdotb_ser = np.dot(mser, input_vecs[1])
            mdotd_ser = 0.5*(mdota_ser + mdotb_ser)
            mdots_ser = mdota_ser - mdotb_ser
            initial_mdotds = [mdotd_ser[0], mdots_ser[0]]  # Should equal epss, epsd
    
            td_grid[0, i, j], ts_grid[0, i, j] = analytical_convergence_times_2d(initial_mdotds, 
                                                            norms_vecds, learning_mu, sigma2_nu)
            td_grid[1, i, j], ts_grid[1, i, j] = find_convergence_time(tser, mdotd_ser, mdots_ser, sigma2_nu)
        print("Completed {} points at eps_d = {}".format(n_tries, epsd))
    
    return np.stack([epsd_axis, epss_axis]), np.stack([td_grid, ts_grid])

def plot_time_analysis_results(eps_axes, time_grids):
    n_tries = eps_axes[0].size
    epsd_axis, epss_axis = eps_axes
    td_grid, ts_grid = time_grids
    
    # Plot results
    fig, axes = plt.subplots(1, 2)
    axes = axes.flatten()
    # td as a function of epsd; should not depend on eps_s
    ax = axes[0]
    colors = sns.color_palette("mako", n_colors=n_tries)
    for j in range(epsd_axis.size):
        ax.plot(epsd_axis, td_grid[1, :, j], color=colors[j], ls="--",
                label=r"$\epsilon_s = {:.3f}$".format(epss_axis[j]), marker="o")
    # Plot theory, independent of eps_s
    ax.plot(epsd_axis, td_grid[0, :, 0], label="Analytical", color="k", ls="-", lw=2.0)
    ax.set(xlabel=r"$\epsilon_d$ (initial $c_d = \vec{m} \cdot \vec{x}_d$)", 
           ylabel=r"$t_d$ ($\vec{m} \cdot \vec{x}_d$ convergence time)", 
           title="First phase", xscale="log", yscale="log")
    ax.legend(title="Simulation", fontsize=9)
    
    # ts - td as a function of epss
    ax = axes[1]
    colors = sns.color_palette("flare", n_colors=n_tries)
    # Plot simulations
    for i in range(epsd_axis.size):
        ax.plot(epss_axis, ts_grid[1, i] - td_grid[1, i], color=colors[i], ls="--",
                label=r"$\epsilon_d = {:.3f}$".format(epsd_axis[i]), marker="o")
    # Plot theory, independent of eps_s
    ax.plot(epss_axis, ts_grid[0, 0] - td_grid[0, 0], label="Analytical", color="k", ls="-", lw=2.0)
    ax.set(xlabel=r"$\epsilon_s$ (initial $c_s = \vec{m} \cdot \vec{x}_s$)", 
           ylabel=r"$t_s$ ($\vec{m} \cdot \vec{x}_s$ convergence time)", 
           title="Second phase", xscale="log", yscale="log")
    ax.legend(title="Simulation", fontsize=9)
    
    fig.set_size_inches(7.5, 3.5)
    fig.tight_layout()
    return fig, axes

In [ ]:
eps_axes, time_grids = scaling_convergence_time(back_components, learnrate_ibcm,
                        tau_avg_ibcm, moments_conc, tau_nu, 
                        seed_sequence=np.random.SeedSequence(0xf06a864ae983b9fa690fa682a26aa05), n_tries=4)

fig, axes = plot_time_analysis_results(eps_axes, time_grids)
# fig.savefig("../figures/two_odors/ibcm_neuron_2d_convergence_time_scaling.pdf", transparent=True)
plt.show()
plt.close()